In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Multi-Symbol Intraday Candlestick Strategy (Backtest + Live)
with Telegram Alerts + CSV Logging + IST timestamps + EMA200 Trend Filter

Patterns:
  - Hammer (bullish reversal)
  - Shooting Star (bearish reversal)
  - Bullish / Bearish Engulfing
  - Inside Bar breakout (up/down)

Features:
  - WATCHLIST of NSE symbols (.NS) or indices (^NSEI, ^NSEBANK)
  - Backtest a specific trading date (5m candles), IST-aware day filter
  - Live loop uses last COMPLETED candle only (alert/paper-trade)
  - Fixed SL/TP, one-position-per-symbol, portfolio cap
  - Reason strings (pattern + context near prev-day H/L + EMA trend tag)
  - Telegram alerts for entries/exits (optional)
  - CSV logging for backtest & live sessions
  - **IST timestamps in CSVs** (entry_time_ist / ts_bar_close_ist)
  - **EMA200 trend filter toggle** (use_ema200)

NOTE: Yahoo intraday can be delayed; treat live loop as alerting/paper mode.
"""

import os, time, logging, warnings, json
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
log = logging.getLogger("px_candle_system")

try:
    import yfinance as yf
except Exception as e:
    raise SystemExit("Please install yfinance: pip install yfinance") from e

# Optional: requests for Telegram; fallback to urllib if unavailable
try:
    import requests  # type: ignore
except Exception:
    requests = None  # we'll fallback


# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # ---- Data & Universe ----
    watchlist: List[str] = field(default_factory=lambda: [
        'ADANIENT.NS','ADANIPORTS.NS','APOLLOHOSP.NS','ASIANPAINT.NS','AXISBANK.NS',
        'BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BEL.NS','BHARTIARTL.NS',
        'CIPLA.NS','COALINDIA.NS','DRREDDY.NS','EICHERMOT.NS','GRASIM.NS',
        'HCLTECH.NS','HDFCBANK.NS','HDFCLIFE.NS','HINDALCO.NS','HINDUNILVR.NS',
        'ICICIBANK.NS','ITC.NS','INFY.NS','INDIGO.NS','JSWSTEEL.NS',
        'JIOFIN.NS','KOTAKBANK.NS','LT.NS','M&M.NS','MARUTI.NS',
        'MAXHEALTH.NS','NTPC.NS','NESTLEIND.NS','ONGC.NS','POWERGRID.NS',
        'RELIANCE.NS','SBILIFE.NS','SHRIRAMFIN.NS','SBIN.NS','SUNPHARMA.NS',
        'TCS.NS','TATACONSUM.NS','TATAMOTORS.NS','TATASTEEL.NS','TECHM.NS',
        'TITAN.NS','TRENT.NS','ULTRACEMCO.NS','WIPRO.NS'
    ])
    interval: str = "5m"                  # '1m' (7 days) or '5m' (60 days)

    # ---- Backtest (single day) ----
    backtest_date: str = "2025-10-16"     # YYYY-MM-DD

    # ---- Live loop ----
    live_mode: bool = False               # Set True to run live
    poll_seconds: int = 40
    market_open_hhmm: str = "09:15"       # IST
    market_close_hhmm: str = "15:30"      # IST

    # ---- Risk / Trade Management ----
    stop_loss_pct: float = 0.003          # 0.30%
    target_pct: float = 0.006             # 0.60%
    capital_inr: float = 200000.0
    capital_per_trade_frac: float = 0.10  # 10% notional per trade for P&L calc
    max_positions_total: int = 10         # portfolio-level cap

    # ---- Filters / Context ----
    use_prev_day_levels: bool = True      # tag signals near prev-day H/L
    use_ema200: bool = True               # *** NEW: toggle EMA trend filter
    ema_len: int = 200                    # EMA length (default 200)

    # ---- Logging / Outputs ----
    outputs_dir: str = "outputs"          # CSVs under outputs/YYYY-MM-DD
    print_first_trades: int = 5

    # ---- Safety ----
    one_position_per_symbol: bool = True  # avoid pyramiding without exits

    # ---- Telegram (optional) ----
    TELEGRAM_BOT_TOKEN: str = ""          # e.g. "123456:ABC-DEF..."
    TELEGRAM_CHAT_ID: str = ""            # e.g. your_chat_id or @channelusername
    TELEGRAM_ENABLED: bool = False        # set True after filling token & chat_id

cfg = Config()


# =========================
# TIMEZONE HELPERS (Robust)
# =========================
def _to_utc_index(idx: pd.DatetimeIndex) -> pd.DatetimeIndex:
    if getattr(idx, "tz", None) is None:
        return idx.tz_localize("UTC")
    return idx.tz_convert("UTC")

def _to_ist_index(idx: pd.DatetimeIndex) -> pd.DatetimeIndex:
    if getattr(idx, "tz", None) is None:
        return idx.tz_localize("UTC").tz_convert("Asia/Kolkata")
    return idx.tz_convert("Asia/Kolkata")

def ist_now() -> pd.Timestamp:
    return pd.Timestamp.utcnow().tz_localize("UTC").tz_convert("Asia/Kolkata")

def _hhmm_to_time(hhmm: str) -> pd.Timestamp:
    hh, mm = map(int, hhmm.split(":"))
    return pd.Timestamp(2000, 1, 1, hh, mm, tz="Asia/Kolkata")

def is_market_open_ist(now_ist: Optional[pd.Timestamp] = None) -> bool:
    now_ist = now_ist or ist_now()
    t = now_ist.tz_localize("Asia/Kolkata") if now_ist.tz is None else now_ist
    open_t = _hhmm_to_time(cfg.market_open_hhmm).time()
    close_t = _hhmm_to_time(cfg.market_close_hhmm).time()
    return open_t <= t.time() <= close_t

def date_range_pad(d: str, days_before=1, days_after=1) -> Tuple[str, str]:
    start = (pd.Timestamp(d) - pd.Timedelta(days=days_before)).strftime("%Y-%m-%d")
    end   = (pd.Timestamp(d) + pd.Timedelta(days=days_after)).strftime("%Y-%m-%d")
    return start, end

def ts_to_ist_str(ts: pd.Timestamp) -> str:
    """Format any timestamp to IST string 'YYYY-MM-DD HH:MM:SS'."""
    if isinstance(ts, str):
        try:
            ts = pd.Timestamp(ts)
        except Exception:
            return str(ts)
    if ts.tz is None:
        ts = ts.tz_localize("UTC")
    return ts.tz_convert("Asia/Kolkata").strftime("%Y-%m-%d %H:%M:%S")


# =========================
# IO / TELEGRAM HELPERS
# =========================
def ensure_outdir_for_day(day: str) -> str:
    out_dir = os.path.join(cfg.outputs_dir, day)
    os.makedirs(out_dir, exist_ok=True)
    return out_dir

def csv_append(path: str, df_row: pd.DataFrame):
    header = not os.path.exists(path)
    df_row.to_csv(path, index=False, mode="a", header=header, encoding="utf-8")

def send_telegram(text: str):
    if not cfg.TELEGRAM_ENABLED:
        return
    if not cfg.TELEGRAM_BOT_TOKEN or not cfg.TELEGRAM_CHAT_ID:
        log.warning("Telegram enabled but BOT_TOKEN/CHAT_ID missing.")
        return
    url = f"https://api.telegram.org/bot{cfg.TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {"chat_id": cfg.TELEGRAM_CHAT_ID, "text": text, "parse_mode": "HTML", "disable_web_page_preview": True}
    try:
        if requests is not None:
            r = requests.post(url, data=payload, timeout=10)
            if r.status_code != 200:
                log.warning(f"Telegram send failed: {r.status_code} {r.text}")
        else:
            import urllib.request, urllib.parse
            data = urllib.parse.urlencode(payload).encode()
            req = urllib.request.Request(url, data=data)
            with urllib.request.urlopen(req, timeout=10) as resp:
                if resp.status != 200:
                    log.warning(f"Telegram send failed (urllib): {resp.status}")
    except Exception as e:
        log.warning(f"Telegram send exception: {e}")


# =========================
# DATA DOWNLOAD
# =========================
def yf_download_multi(symbols: List[str], start: str, end: str, interval: str) -> Dict[str, pd.DataFrame]:
    if not symbols:
        return {}
    data = yf.download(
        tickers=" ".join(symbols), start=start, end=end, interval=interval,
        auto_adjust=False, group_by="ticker", progress=False, threads=True
    )

    out: Dict[str, pd.DataFrame] = {}
    if isinstance(data, pd.DataFrame) and isinstance(data.columns, pd.MultiIndex):
        for sym in symbols:
            if sym in data.columns.get_level_values(0):
                df = data[sym].copy()
                if not df.empty:
                    df = df.rename(columns=str.title).dropna()
                    out[sym] = df
    elif isinstance(data, pd.DataFrame) and not data.empty:
        sym = symbols[0]
        out[sym] = data.rename(columns=str.title).dropna()
    return out


# =========================
# CONTEXT: PREV-DAY LEVELS
# =========================
def attach_prev_day_levels(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    idx_ist = _to_ist_index(df.index)
    day_ist = pd.to_datetime(idx_ist.date)
    out = df.copy()
    out["__day_ist"] = day_ist
    daily = out.groupby("__day_ist").agg(PrevHigh=("High", "max"),
                                         PrevLow=("Low", "min"))
    daily["PrevHigh_shift"] = daily["PrevHigh"].shift(1)
    daily["PrevLow_shift"]  = daily["PrevLow"].shift(1)
    out = out.join(daily[["PrevHigh_shift", "PrevLow_shift"]], on="__day_ist")
    out = out.rename(columns={"PrevHigh_shift": "PrevDayHigh",
                              "PrevLow_shift": "PrevDayLow"})
    return out


# =========================
# PATTERN DETECTION + EMA
# =========================
def add_ema(df: pd.DataFrame) -> pd.DataFrame:
    """Add EMA column for trend filter."""
    if df.empty:
        return df
    out = df.copy()
    out[f"EMA{cfg.ema_len}"] = out["Close"].ewm(span=cfg.ema_len, adjust=False, min_periods=cfg.ema_len).mean()
    return out

def detect_patterns(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    o, h, l, c = df["Open"], df["High"], df["Low"], df["Close"]
    body = (c - o).abs().clip(lower=1e-9)
    real_body_top = np.maximum(o, c)
    real_body_bottom = np.minimum(o, c)
    upper_wick = h - real_body_top
    lower_wick = real_body_bottom - l

    # Hammer (bullish)
    df["is_hammer"] = (lower_wick >= 2 * body) & (upper_wick <= 0.5 * body) & (c > o)

    # Shooting Star (bearish)
    df["is_shooting_star"] = (upper_wick >= 2 * body) & (lower_wick <= 0.5 * body) & (c < o)

    # Engulfing
    prev_o, prev_c = o.shift(1), c.shift(1)
    df["is_bullish_engulf"] = (c > o) & (prev_c < prev_o) & (c >= prev_o) & (o <= prev_c)
    df["is_bearish_engulf"] = (c < o) & (prev_c > prev_o) & (c <= prev_o) & (o >= prev_c)

    # Inside Bar
    prev_h, prev_l = h.shift(1), l.shift(1)
    df["is_inside_bar"] = (h <= prev_h) & (l >= prev_l)
    return df


# =========================
# SIGNALS (with EMA filter)
# =========================
def reason_text(row: pd.Series, prev: Optional[pd.Series]) -> str:
    parts = []
    if row.get("is_hammer"): parts.append("Hammer")
    if row.get("is_shooting_star"): parts.append("ShootingStar")
    if row.get("is_bullish_engulf"): parts.append("BullishEngulf")
    if row.get("is_bearish_engulf"): parts.append("BearishEngulf")
    if prev is not None and prev.get("is_inside_bar"):
        if row["Close"] > prev["High"]:
            parts.append("InsideBreakUp")
        elif row["Close"] < prev["Low"]:
            parts.append("InsideBreakDown")
    if cfg.use_prev_day_levels:
        pH = row.get("PrevDayHigh", np.nan); pL = row.get("PrevDayLow", np.nan)
        if not pd.isna(pH) and abs((row["Close"] - pH) / pH) < 0.002:
            parts.append("nearPrevDayHigh")
        if not pd.isna(pL) and abs((row["Close"] - pL) / pL) < 0.002:
            parts.append("nearPrevDayLow")
    # EMA tag
    ema = row.get(f"EMA{cfg.ema_len}", np.nan)
    if cfg.use_ema200 and not pd.isna(ema):
        if row["Close"] >= ema:
            parts.append(f"EMA{cfg.ema_len}Up")
        else:
            parts.append(f"EMA{cfg.ema_len}Down")
    return "+".join(parts) if parts else "Pattern/Context"

def generate_signals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["signal"] = np.nan
    df["signal_reason"] = ""
    ema_col = f"EMA{cfg.ema_len}" if cfg.use_ema200 else None

    for i in range(1, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i - 1]
        sig = None

        # Base pattern logic
        if row["is_hammer"] or row["is_bullish_engulf"]:
            sig = "BUY"
        elif row["is_shooting_star"] or row["is_bearish_engulf"]:
            sig = "SELL"
        elif prev["is_inside_bar"]:
            if row["Close"] > prev["High"]:
                sig = "BUY"
            elif row["Close"] < prev["Low"]:
                sig = "SELL"

        # EMA trend filter
        if sig and cfg.use_ema200 and ema_col in df.columns and not pd.isna(row[ema_col]):
            if sig == "BUY" and not (row["Close"] >= row[ema_col]):
                sig = None  # reject longs below EMA
            elif sig == "SELL" and not (row["Close"] <= row[ema_col]):
                sig = None  # reject shorts above EMA

        if sig:
            df.loc[df.index[i], "signal"] = sig
            df.loc[df.index[i], "signal_reason"] = reason_text(row, prev)

    return df


# =========================
# BACKTEST (SINGLE DAY)
# =========================
def backtest_single_day(symbols: List[str], date: str, interval: str):
    start, end = date_range_pad(date, 1, 1)
    frames: Dict[str, pd.DataFrame] = yf_download_multi(symbols, start, end, interval)

    all_trades = []
    for sym, df in frames.items():
        df = df.copy().sort_index()
        if df.empty:
            continue

        # Filter to chosen IST day
        df_ist_index = _to_ist_index(df.index)
        df["__date_ist"] = pd.to_datetime(df_ist_index.date)
        wanted = pd.Timestamp(date)
        df = df[df["__date_ist"] == wanted]
        if df.empty:
            continue

        if cfg.use_prev_day_levels:
            df = attach_prev_day_levels(df)
        # Add EMA and patterns
        df = add_ema(df)
        df = detect_patterns(df)
        df = generate_signals(df)

        # One-position-per-symbol simulator using next-bar-open entries
        position = None
        for i in range(len(df) - 1):
            row = df.iloc[i]
            nxt = df.iloc[i + 1]
            if position is None:
                if row["signal"] == "BUY":
                    entry = float(nxt["Open"])
                    sl = entry * (1 - cfg.stop_loss_pct)
                    tp = entry * (1 + cfg.target_pct)
                    position = dict(symbol=sym, side="LONG", entry=entry, sl=sl, tp=tp,
                                    reason=row["signal_reason"], entry_time=nxt.name)
                elif row["signal"] == "SELL":
                    entry = float(nxt["Open"])
                    sl = entry * (1 + cfg.stop_loss_pct)
                    tp = entry * (1 - cfg.target_pct)
                    position = dict(symbol=sym, side="SHORT", entry=entry, sl=sl, tp=tp,
                                    reason=row["signal_reason"], entry_time=nxt.name)
            else:
                h, l = float(row["High"]), float(row["Low"])
                if position["side"] == "LONG":
                    if l <= position["sl"]:
                        pnl = (position["sl"] - position["entry"]) / position["entry"]
                        all_trades.append({**position, "exit": position["sl"], "pnl": pnl, "exit_reason": "SL"})
                        position = None
                    elif h >= position["tp"]:
                        pnl = (position["tp"] - position["entry"]) / position["entry"]
                        all_trades.append({**position, "exit": position["tp"], "pnl": pnl, "exit_reason": "TP"})
                        position = None
                else:
                    if h >= position["sl"]:
                        pnl = (position["entry"] - position["sl"]) / position["entry"]
                        all_trades.append({**position, "exit": position["sl"], "pnl": pnl, "exit_reason": "SL"})
                        position = None
                    elif l <= position["tp"]:
                        pnl = (position["entry"] - position["tp"]) / position["entry"]
                        all_trades.append({**position, "exit": position["tp"], "pnl": pnl, "exit_reason": "TP"})
                        position = None

    trades = pd.DataFrame(all_trades)
    # Save CSV + summary JSON (with IST timestamps)
    out_dir = ensure_outdir_for_day(date)
    if trades.empty:
        summary = dict(date=date, n_trades=0, win_rate_pct=0.0, profitable_trades=0,
                       avg_pnl_pct=0.0, total_pnl_inr=0.0)
        trades.to_csv(os.path.join(out_dir, "backtest_trades.csv"), index=False)
        with open(os.path.join(out_dir, "backtest_summary.json"), "w") as f:
            json.dump(summary, f, indent=2)
        return trades, summary

    # enrich with IST strings
    trades = trades.sort_values("entry_time").reset_index(drop=True)
    trades["entry_time_ist"] = trades["entry_time"].apply(ts_to_ist_str)

    notion = cfg.capital_inr * cfg.capital_per_trade_frac
    trades["pnl_inr"] = trades["pnl"] * notion
    win_rate = float((trades["pnl"] > 0).mean() * 100.0)
    profitable = int((trades["pnl"] > 0).sum())
    summary = dict(
        date=date,
        n_trades=int(len(trades)),
        win_rate_pct=round(win_rate, 2),
        profitable_trades=profitable,
        avg_pnl_pct=round(float(trades["pnl"].mean() * 100.0), 3),
        total_pnl_inr=round(float(trades["pnl_inr"].sum()), 2),
    )
    trades.to_csv(os.path.join(out_dir, "backtest_trades.csv"), index=False)
    with open(os.path.join(out_dir, "backtest_summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    return trades, summary


# =========================
# LIVE LOOP (ALERT/PAPER + CSV, IST fields)
# =========================
def floor_to_prev_completed(ts_utc: pd.Timestamp, interval: str) -> pd.Timestamp:
    if ts_utc.tz is None:
        ts_utc = ts_utc.tz_localize("UTC")
    minutes = int(interval.replace("m", "")) if interval.endswith("m") else 5
    floored = ts_utc - pd.Timedelta(
        minutes=ts_utc.minute % minutes,
        seconds=ts_utc.second,
        microseconds=ts_utc.microsecond
    )
    return floored

def live_loop():
    log.info(f"Starting LIVE loop | interval={cfg.interval} | symbols={len(cfg.watchlist)}")
    positions: Dict[str, Optional[Dict]] = {}   # sym -> position or None
    last_bar_seen: Dict[str, pd.Timestamp] = {}

    today_str = ist_now().strftime("%Y-%m-%d")
    out_dir = ensure_outdir_for_day(today_str)
    live_csv = os.path.join(out_dir, "live_trades.csv")

    while True:
        now_ist = ist_now()
        if not is_market_open_ist(now_ist):
            log.info("Market not open (IST). Sleeping 60s.")
            time.sleep(60)
            continue

        start = (pd.Timestamp.utcnow() - pd.Timedelta(days=2)).strftime("%Y-%m-%d")
        end   = (pd.Timestamp.utcnow() + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
        data = yf_download_multi(cfg.watchlist, start, end, cfg.interval)

        for sym, df in data.items():
            if df.empty:
                continue

            if cfg.use_prev_day_levels:
                df = attach_prev_day_levels(df)
            # EMA + patterns + signals
            df = add_ema(df)
            df = detect_patterns(df)
            df = generate_signals(df)

            df = df.copy()
            df.index = _to_utc_index(df.index)

            now_utc = pd.Timestamp.utcnow().tz_localize("UTC")
            last_completed_close = floor_to_prev_completed(now_utc, cfg.interval)
            bar = df[df.index <= last_completed_close].tail(1)
            if bar.empty:
                continue

            bar_ts = bar.index[-1]
            if last_bar_seen.get(sym) == bar_ts:
                continue
            last_bar_seen[sym] = bar_ts

            row = bar.iloc[-1]
            signal = row.get("signal")
            reason = row.get("signal_reason", "")
            pos = positions.get(sym)

            # ENTRY
            if signal and (pos is None) and (sum(p is not None for p in positions.values()) < cfg.max_positions_total):
                entry = float(row["Close"])  # alert-level entry; realistic trading: next open
                if signal == "BUY":
                    sl = entry * (1 - cfg.stop_loss_pct)
                    tp = entry * (1 + cfg.target_pct)
                    positions[sym] = dict(side="LONG", entry=entry, sl=sl, tp=tp,
                                          entry_time=str(bar_ts), reason=reason)
                    msg = f"🟢 [ENTRY] {sym} LONG @~{entry:.2f}\nSL {sl:.2f} | TP {tp:.2f}\n{reason}"
                else:
                    sl = entry * (1 + cfg.stop_loss_pct)
                    tp = entry * (1 - cfg.target_pct)
                    positions[sym] = dict(side="SHORT", entry=entry, sl=sl, tp=tp,
                                          entry_time=str(bar_ts), reason=reason)
                    msg = f"🔴 [ENTRY] {sym} SHORT @~{entry:.2f}\nSL {sl:.2f} | TP {tp:.2f}\n{reason}"

                log.info(msg.replace("\n", " | "))
                send_telegram(msg)

                # CSV append entry (IST field included)
                row_df = pd.DataFrame([{
                    "ts_bar_close_utc": str(bar_ts),
                    "ts_bar_close_ist": ts_to_ist_str(bar_ts),
                    "symbol": sym,
                    "event": "ENTRY",
                    "side": positions[sym]["side"],
                    "price": entry,
                    "sl": sl,
                    "tp": tp,
                    "reason": reason
                }])
                csv_append(live_csv, row_df)

            # EXIT mgmt on just-closed bar range
            pos = positions.get(sym)
            if pos:
                hi, lo = float(row["High"]), float(row["Low"])
                if pos["side"] == "LONG":
                    if lo <= pos["sl"]:
                        pnl = (pos["sl"] - pos["entry"]) / pos["entry"]
                        msg = f"⚠️ [EXIT SL] {sym} LONG @ {pos['sl']:.2f} | PnL {pnl*100:.2f}%\n{pos['reason']}"
                        log.info(msg.replace("\n", " | "))
                        send_telegram(msg)
                        positions[sym] = None
                        csv_append(live_csv, pd.DataFrame([{
                            "ts_bar_close_utc": str(bar_ts),
                            "ts_bar_close_ist": ts_to_ist_str(bar_ts),
                            "symbol": sym, "event": "EXIT_SL", "side": "LONG",
                            "price": pos["sl"], "pnl_pct": pnl*100.0, "reason": pos["reason"]
                        }]))
                    elif hi >= pos["tp"]:
                        pnl = (pos["tp"] - pos["entry"]) / pos["entry"]
                        msg = f"✅ [EXIT TP] {sym} LONG @ {pos['tp']:.2f} | PnL {pnl*100:.2f}%\n{pos['reason']}"
                        log.info(msg.replace("\n", " | "))
                        send_telegram(msg)
                        positions[sym] = None
                        csv_append(live_csv, pd.DataFrame([{
                            "ts_bar_close_utc": str(bar_ts),
                            "ts_bar_close_ist": ts_to_ist_str(bar_ts),
                            "symbol": sym, "event": "EXIT_TP", "side": "LONG",
                            "price": pos["tp"], "pnl_pct": pnl*100.0, "reason": pos["reason"]
                        }]))
                else:
                    if hi >= pos["sl"]:
                        pnl = (pos["entry"] - pos["sl"]) / pos["entry"]
                        msg = f"⚠️ [EXIT SL] {sym} SHORT @ {pos['sl']:.2f} | PnL {pnl*100:.2f}%\n{pos['reason']}"
                        log.info(msg.replace("\n", " | "))
                        send_telegram(msg)
                        positions[sym] = None
                        csv_append(live_csv, pd.DataFrame([{
                            "ts_bar_close_utc": str(bar_ts),
                            "ts_bar_close_ist": ts_to_ist_str(bar_ts),
                            "symbol": sym, "event": "EXIT_SL", "side": "SHORT",
                            "price": pos["sl"], "pnl_pct": pnl*100.0, "reason": pos["reason"]
                        }]))
                    elif lo <= pos["tp"]:
                        pnl = (pos["entry"] - pos["tp"]) / pos["entry"]
                        msg = f"✅ [EXIT TP] {sym} SHORT @ {pos['tp']:.2f} | PnL {pnl*100:.2f}%\n{pos['reason']}"
                        log.info(msg.replace("\n", " | "))
                        send_telegram(msg)
                        positions[sym] = None
                        csv_append(live_csv, pd.DataFrame([{
                            "ts_bar_close_utc": str(bar_ts),
                            "ts_bar_close_ist": ts_to_ist_str(bar_ts),
                            "symbol": sym, "event": "EXIT_TP", "side": "SHORT",
                            "price": pos["tp"], "pnl_pct": pnl*100.0, "reason": pos["reason"]
                        }]))

        time.sleep(cfg.poll_seconds)


# =========================
# MAIN
# =========================
def main():
    # BACKTEST
    trades, summary = backtest_single_day(cfg.watchlist, cfg.backtest_date, cfg.interval)
    log.info("===== Backtest Summary =====")
    for k, v in summary.items():
        log.info(f"{k}: {v}")
    if not trades.empty:
        log.info("Sample trades:")
        print(trades.head(cfg.print_first_trades)[
            ["symbol","side","entry_time","entry_time_ist","entry","exit","exit_reason","pnl","reason"]
        ])
        log.info(f"Profitable trades: {summary['profitable_trades']} out of {summary['n_trades']} "
                 f"({summary['win_rate_pct']}% win rate)")
        # Telegram summary (optional)
        if cfg.TELEGRAM_ENABLED:
            send_telegram(
                f"📊 Backtest {summary['date']}\n"
                f"Trades: {summary['n_trades']}\n"
                f"Profitable: {summary['profitable_trades']}\n"
                f"Win%: {summary['win_rate_pct']} | AvgPnL%: {summary['avg_pnl_pct']}\n"
                f"Total PnL (₹): {summary['total_pnl_inr']}\n"
                f"EMA Filter: {'ON' if cfg.use_ema200 else 'OFF'} (len={cfg.ema_len})"
            )
    else:
        log.info("No trades on this date.")

    # LIVE (optional)
    if cfg.live_mode:
        live_loop()


if __name__ == "__main__":
    main()


2025-10-19 18:45:01,658 | INFO | ===== Backtest Summary =====
2025-10-19 18:45:01,659 | INFO | date: 2025-10-16
2025-10-19 18:45:01,659 | INFO | n_trades: 131
2025-10-19 18:45:01,659 | INFO | win_rate_pct: 31.3
2025-10-19 18:45:01,659 | INFO | profitable_trades: 41
2025-10-19 18:45:01,660 | INFO | avg_pnl_pct: -0.018
2025-10-19 18:45:01,660 | INFO | total_pnl_inr: -480.0
2025-10-19 18:45:01,660 | INFO | Sample trades:
2025-10-19 18:45:01,665 | INFO | Profitable trades: 41 out of 131 (31.3% win rate)


          symbol   side                entry_time       entry_time_ist  \
0    HINDALCO.NS   LONG 2025-10-16 03:55:00+00:00  2025-10-16 09:25:00   
1  HINDUNILVR.NS  SHORT 2025-10-16 03:55:00+00:00  2025-10-16 09:25:00   
2    HDFCBANK.NS   LONG 2025-10-16 03:55:00+00:00  2025-10-16 09:25:00   
3   COALINDIA.NS  SHORT 2025-10-16 03:55:00+00:00  2025-10-16 09:25:00   
4       WIPRO.NS  SHORT 2025-10-16 03:55:00+00:00  2025-10-16 09:25:00   

         entry         exit exit_reason    pnl         reason  
0   767.750000   772.356500          TP  0.006  BullishEngulf  
1  2524.699951  2509.551751          TP  0.006   ShootingStar  
2   983.200012   989.099212          TP  0.006  BullishEngulf  
3   385.700012   386.857112          SL -0.003   ShootingStar  
4   250.250000   251.000750          SL -0.003  BearishEngulf  
